For $\Omega_{3D}$,

$$
\Delta p_{3D} = \beta (\overline{p_{3D}} - p_{1D}) \delta_{\Lambda} + f_{3D}
$$

$$
p_{3D} = \text{constant on } \partial \Omega_{3D}.
$$

For $\Omega_{1D}$,

$$
\frac{\partial^2 p_{1D}}{\partial s^2} = -\beta (\overline{p_{3D}} - p_{1D}) + f_{1D}
$$
$$
- \frac{\partial p_{1D}}{\partial s} \Bigg|_{s = L} = \gamma (p_{\infty} - p_{1D}(L))
$$

$$
- \frac{\partial p_{1D}}{\partial s} \Bigg|_{s = 0} = \gamma (p_{\infty} - p_{1D}(0)).
$$

For now, assume homogeneity so $f_{1D} = 0$ and $f_{3D} = 0$.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# @title Install required libraries
%%capture
!pip3 install virtualenv

import sys
WD_PATH = "/content/drive/MyDrive/Research/3D-1D/"
VENV_PATH = WD_PATH + "/.venv"
sys.path.append(WD_PATH + "modules")

import os, re
if not os.path.exists(VENV_PATH):
  !virtualenv $VENV_PATH

sys.path.append(VENV_PATH + "/lib/python3.10/site-packages")

def replace_in_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    # Replace 'ufl' with 'ufl_legacy'
    content = re.sub(r'\bufl\b', 'ufl_legacy', content)

    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(content)

def process_directory(directory):
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.py'):
                file_path = os.path.join(root, file)
                replace_in_file(file_path)

# dolfin
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

# block
try:
    import block
except ImportError:
    !git clone https://bitbucket.org/fenics-apps/cbc.block/src/master/
    !source $VENV_PATH/bin/activate && python3 -m pip install master/

# fenics_ii
try:
    import xii
except ImportError:
    !git clone https://github.com/MiroK/fenics_ii
    process_directory("fenics_ii/")
    !source $VENV_PATH/bin/activate && python3 -m pip install fenics_ii/

# vtk
try:
    import vtk
except ImportError:
    !source $VENV_PATH/bin/activate && pip install vtk

# graphnics
try:
    import graphnics
except ImportError:
    !git clone https://github.com/IngeborgGjerde/graphnics
    !source $VENV_PATH/bin/activate && python3 -m pip install graphnics/

Focus on 1D coupling. Let the domain be a single segment $ [0, L] \in \Omega_{1D} $.

The strong form is
$$ -\frac{d^2 p}{ds^2} + \beta p = \beta \bar{p}_{3D} + f(s). $$

We have
$$ -\frac{dp}{ds}(0) + \gamma p(0) = \gamma p_{\infty}, $$
$$ \frac{dp}{ds}(L) + \gamma p(L) = \gamma p_{\infty}, $$
where $ \gamma = 1.0 $ is the boundary condition coefficient and $ p_{\infty} = 0.5 $ is the far-field pressure.

Using the finite difference method, we discretize the domain into $ N $ grid points with spacing $ \Delta s = \frac{L}{N-1} $. Then the second derivative

$$ \frac{d^2 p}{ds^2} \approx \frac{p_{i-1} - 2p_i + p_{i+1}}{\Delta s^2}. $$

The discretized PDE at an interior point $ s_i $ (for $ 1 \leq i \leq N-2 $) becomes
$$ -\frac{p_{i-1} - 2p_i + p_{i+1}}{\Delta s^2} + \beta p_i = \beta \bar{p}_{3D} + f_i. $$

$$ p_{i-1} - (2 + \beta \Delta s^2)p_i + p_{i+1} = -\beta \Delta s^2 \bar{p}_{3D} - \Delta s^2 f_i. $$

We have at $ s = 0 $
$$ -\frac{p_1 - p_0}{\Delta s} + \gamma p_0 = \gamma p_{\infty}, $$

$$ -p_1 + (-2 - \beta \Delta s^2 - 2\gamma \Delta s)p_0 = -\beta \Delta s^2 \bar{p}_{3D} - 2\gamma \Delta s p_{\infty}. $$

and at $ s = L $
$$ \frac{p_{N-1} - p_{N-2}}{\Delta s} + \gamma p_{N-1} = \gamma p_{\infty}, $$

$$ -p_{N-2} + (-2 - \beta \Delta s^2 - 2\gamma \Delta s)p_{N-1} = -\beta \Delta s^2 \bar{p}_{3D} - 2\gamma \Delta s p_{\infty}. $$

We now form the system $ \mathbf{A}\mathbf{x} = \mathbf{b} $.

$$ \mathbf{A} = \begin{bmatrix}
-2 - \beta \Delta s^2 - 2\gamma \Delta s & 2 & 0 & 0 & \cdots & 0 & 0 \\
1 & -2 - \beta \Delta s^2 & 1 & 0 & \cdots & 0 & 0 \\
0 & 1 & -2 - \beta \Delta s^2 & 1 & \cdots & 0 & 0 \\
0 & 0 & 1 & -2 - \beta \Delta s^2 & \cdots & 0 & 0 \\
\vdots & \vdots & \vdots & \vdots & \ddots & \vdots & \vdots \\
0 & 0 & 0 & 0 & \cdots & 1 & -2 - \beta \Delta s^2 \\
0 & 0 & 0 & 0 & \cdots & 2 & -2 - \beta \Delta s^2 - 2\gamma \Delta s \\
\end{bmatrix}. $$

$$ \mathbf{b} = \begin{bmatrix}
-\beta \Delta s^2 \bar{p}_{3D} - 2\gamma \Delta s p_{\infty} - \Delta s^2 f_0 \\
-\beta \Delta s^2 \bar{p}_{3D} - \Delta s^2 f_1 \\
-\beta \Delta s^2 \bar{p}_{3D} - \Delta s^2 f_2 \\
\vdots \\
-\beta \Delta s^2 \bar{p}_{3D} - \Delta s^2 f_{N-3} \\
-\beta \Delta s^2 \bar{p}_{3D} - \Delta s^2 f_{N-2} \\
-\beta \Delta s^2 \bar{p}_{3D} - 2\gamma \Delta s p_{\infty} - \Delta s^2 f_{N-1} \\
\end{bmatrix}. $$

In [ ]:
# @title Solve 1D PDE by finite difference method
import numpy as np
import matplotlib.pyplot as plt

L = 1.0         # Domain length
beta = 1.0       # Coefficient
p_bar_3D = 1.0   # Mean 3D pressure
p_inf = 0.5     # Far-field pressure
gamma = 1.0      # Boundary condition coefficient
f_1D = lambda s: 0.0  # Source term

N = 100         # Number of grid points
ds = L / (N - 1)

s = np.linspace(0, L, N)

# pressure and coefficient matrix
p_1D = np.zeros(N)
A = np.zeros((N, N))

for i in range(1, N - 1):
    A[i, i - 1] = 1
    A[i, i] = -2 - beta * ds**2
    A[i, i + 1] = 1

# Apply Robin boundary
A[0, 0] = -2 - beta * ds**2 - 2 * gamma * ds
A[0, 1] = 2
A[-1, -2] = 2
A[-1, -1] = -2 - beta * ds**2 - 2 * gamma * ds

# right-hand side vector
b = np.zeros(N)
for i in range(1, N - 1):
    b[i] = -beta * ds**2 * p_bar_3D - ds**2 * f_1D(s[i])
b[0] = -beta * ds**2 * p_bar_3D - 2 * gamma * ds * p_inf - ds**2 * f_1D(s[0])
b[-1] = -beta * ds**2 * p_bar_3D - 2 * gamma * ds * p_inf - ds**2 * f_1D(s[-1])

p_1D = np.linalg.solve(A, b)

plt.figure()
plt.plot(s, p_1D, marker='o', linestyle='-')
plt.xlabel('s')
plt.ylabel('p_1D')
plt.title('1D numpy solution')
plt.grid(True)
plt.legend()
plt.show()

The blocks that we are concerned about are

$$
a_{11} = \alpha_{1} \int_{\Omega_{1D}} \frac{\partial u_{1D}}{\partial s} \frac{\partial v_{1D}}{\partial s} \, ds + \beta \int_{\Omega_{1D}} u_{1D} v_{1D} \, ds + \gamma v_{1D}(L) u_{1D}(L) + \gamma v_{1D}(0) u_{1D}(0)
$$

$$
L_{1} = \beta \int_{\Omega_{1D}} \overline{p_{3D}} v_{1D} \, ds + \gamma v_{1D}(L) p_{\infty} + \gamma v_{1D}(0) p_{\infty}
$$
which account for one vessel segment. ds automatically evaluates at both boundaries so we only need to include one command.

In [ ]:
# @title Solve 1D PDE by FEniCS without modularization
from fenics import *
import matplotlib.pyplot as plt

L = 1.0         # Domain length
beta = 1.0      # Coefficient
p_bar_3D = 1.0  # Mean 3D pressure
p_inf = 0.5     # Far-field pressure
gamma = 1.0     # Boundary condition coefficient
f_1D = Constant(0.0)  # Source term

# mesh and function space
mesh = IntervalMesh(50, 0, L)
V = FunctionSpace(mesh, 'P', 1)

# variational problem
p = TrialFunction(V)
v = TestFunction(V)

a = dot(grad(p), grad(v))*dx + beta*p*v*dx
L = beta*p_bar_3D*v*dx + f_1D*v*dx

# Robin boundary conditions
a += gamma*p*v*ds
L += gamma*p_inf*v*ds

p_1D = Function(V)
solve(a == L, p_1D)

plot(p_1D)
plt.xlabel('s')
plt.ylabel('p_{1D}')
plt.title('1D FEniCS solution')
plt.grid(True)
plt.show()

In [ ]:
# @title Solve 1D PDE by FEniCS with modularization
from fenics import *
import matplotlib.pyplot as plt

L = 1.0         # Domain length
beta = 1.0      # Coefficient
p_bar_3D = 1.0  # Mean 3D pressure
p_inf = 0.5     # Far-field pressure
gamma = 1.0     # Boundary condition coefficient
f_1D = Constant(0.0)  # Source term

# mesh and function space
mesh = IntervalMesh(50, 0, L)
V = FunctionSpace(mesh, 'P', 1)

# Robin boundary condition modularized
class RobinBC(UserExpression):
    def __init__(self, gamma, p_infty, **kwargs):
        super().__init__(**kwargs)
        self.gamma = gamma
        self.p_infty = p_infty

    def eval(self, values, x):
        print(f"eval called! self: {self} values: {values} x: {x}")
        values[0] = self.gamma * self.p_infty

    def value_shape(self):
        return ()

robin_bc = RobinBC(gamma, p_inf, degree=0)

# variational problem
p = TrialFunction(V)
v = TestFunction(V)
a = dot(grad(p), grad(v))*dx + beta*p*v*dx
L = beta*p_bar_3D*v*dx + f_1D*v*dx

# Robin boundary conditions
a += gamma*p*v*ds
L += robin_bc*v*ds

p_1D = Function(V)
solve(a == L, p_1D)

plot(p_1D)
plt.xlabel('s')
plt.ylabel('p_{1D}')
plt.title('1D FEniCS solution')
plt.grid(True)
plt.show()

In [ ]:
# @title Define G = serena data graph, mesh1d = G.mesh
import json
import pandas as pd
from dolfin import *
import numpy as np
import networkx as nx
from graphnics import FenicsGraph

# load JSON data into a DataFrame
def json_to_dataframe(file_path):
    with open(file_path) as f:
        data = json.load(f)
    points = data['markups'][0]['controlPoints']
    radius = data['markups'][0]['measurements'][3]['controlPointValues']
    coords = [point['position'] for point in points]
    df = pd.DataFrame(coords, columns=['x', 'y', 'z'])
    df['radius'] = radius
    return df

# convert all JSON files into df and store them in an array
dataframes = []
for n in range(29):
    file_name = WD_PATH + '/data/pv_json1_mod/Centerline_' + str(n) + '.mrk.json'
    df = json_to_dataframe(file_name)
    dataframes.append(df)

G = FenicsGraph()
ind = 0
branch_points = {}

for n, df in enumerate(dataframes):
    points = df[['x', 'y', 'z']].values.tolist()
    radius = df['radius'].tolist()
    G.add_nodes_from(range(ind - n, ind + len(points) - n))

    v1 = 0
    for key, val in branch_points.items():
        if points[0] == val:
            v1 = key
            break

    v2 = ind - n + 1
    pos_v1 = points[0]
    pos_v2 = points[1]
    G.nodes[v1]["pos"] = pos_v1
    G.nodes[v2]["pos"] = pos_v2
    G.nodes[v1]["radius"] = radius[0]
    G.nodes[v2]["radius"] = radius[1]
    G.add_edge(v1, v2)

    for i in range(len(points) - 2):
        v1 = ind - n + 1 + i
        v2 = v1 + 1
        pos_v1 = points[i + 1]
        pos_v2 = points[i + 2]
        G.nodes[v1]["pos"] = pos_v1
        G.nodes[v2]["pos"] = pos_v2
        G.nodes[v1]["radius"] = radius[i + 1]
        G.nodes[v2]["radius"] = radius[i + 2]
        G.add_edge(v1, v2)

    ind += len(points)
    branch_points.update({ind - n - 1: pos_v2})

# create 1D mesh
G.make_mesh()
mesh1d = G.mesh

# get positions of 1d mesh
pos = nx.get_node_attributes(G, "pos")
node_coords = np.asarray(list(pos.values()))
xmin, ymin, zmin = np.min(node_coords, axis = 0)
radii = nx.get_node_attributes(G, "radius")

# recenter 1d coordinates to all be >= 0
d = mesh1d.coordinates()
d[:, :] += [-xmin, -ymin, -zmin]

In [ ]:
# figure out how to import it correctly
# %run /content/drive/MyDrive/Research/3D-1D/modules/visualizer.py

#@title Visualizer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import scipy
from dolfin import *

def visualize(mesh1d, uh1d, mesh3d=None, uh3d=None, z_level=50, elev=10, azim=30, boundary_list=None):
    node_coords = mesh1d.coordinates()
    pressure_values = uh1d.compute_vertex_values(mesh1d)

    fig, ax1 = plt.subplots(
        1,  # rows
        2 if (uh3d is not None and mesh3d is not None) else 1,  # columns
        figsize=(20, 8) if (uh3d is not None and mesh3d is not None) else (10, 8)  # fig size
    )

    if uh3d is not None and mesh3d is not None:
        pressure_3d_values = uh3d.compute_vertex_values(mesh3d)
        coords_3d = mesh3d.coordinates()

        z_coords = coords_3d[:, 2]
        mask = np.isclose(z_coords, z_level, atol=mesh3d.hmin())
        filtered_coords = coords_3d[mask]
        filtered_pressure = pressure_3d_values[mask]

        # grid for the heatmap
        x = filtered_coords[:, 0]
        y = filtered_coords[:, 1]
        z = filtered_pressure
        grid_size = 100
        xi = np.linspace(x.min(), x.max(), grid_size)
        yi = np.linspace(y.min(), y.max(), grid_size)
        zi = scipy.interpolate.griddata((x, y), z, (xi[None, :], yi[:, None]), method='linear')

        ax1 = fig.add_subplot(121, projection='3d')
        fig.get_axes()[0].axis('off')
        fig.get_axes()[1].axis('off')
    else:
        ax1 = fig.add_subplot(111, projection='3d')
        fig.get_axes()[0].axis('off')

    # 2D plane at Z = z_level
    x_range = np.linspace(ax1.get_xlim()[0], ax1.get_xlim()[1], num=10)
    y_range = np.linspace(ax1.get_ylim()[0], ax1.get_ylim()[1], num=10)
    x_plane, y_plane = np.meshgrid(x_range, y_range)
    z_plane = np.full(x_plane.shape, z_level)
    ax1.plot_surface(x_plane, y_plane, z_plane, color='m', alpha=0.3)

    ax1.view_init(elev=elev, azim=azim)
    sc = ax1.scatter(node_coords[:, 0], node_coords[:, 1], node_coords[:, 2], c=pressure_values, cmap='viridis', s=0.1)
    cbar = plt.colorbar(sc, ax=ax1)
    cbar.set_label('1D Pressure')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title('1D Domain')

    if boundary_list is not None:
        for point in boundary_list:
            ax1.scatter(*point, color='red')

    # 2D heatmap subplot
    if uh3d is not None:
        ax2 = fig.add_subplot(122)
        cf = ax2.contourf(xi, yi, zi, levels=100, cmap='viridis')
        cbar = plt.colorbar(cf, ax=ax2)
        cbar.set_label('Pressure')
        ax2.set_xlabel('X')
        ax2.set_ylabel('Y')
        ax2.set_title(f'3D Domain Section at Z = {z_level}')

    print(fig.get_axes())
    plt.show()

In [ ]:
# @title 1D domain analysis only with Dirichlet boundaries
# function space
V1 = FunctionSpace(mesh1d, "CG", 1)
u1 = TrialFunction(V1)
v1 = TestFunction(V1)

# constants for simulation
alpha1 = Constant(1.45e4)
beta = Constant(3.09e-5)

# Dirichlet boundary condition
bc_1d = Expression("0.02*x[2]+6", degree=0)

# measure for 1D mesh
dxGamma = Measure("dx", domain=mesh1d)

# blocks
a11 = alpha1 * inner(grad(u1), grad(v1)) * dx + beta * inner(u1, v1) * dxGamma

# right-hand side
L1 = inner(Constant(0), v1) * dxGamma

# assemble system
A, b = assemble_system(a11, L1, DirichletBC(V1, bc_1d, "on_boundary"))

# solve
uh1d = Function(V1)
solve(A, uh1d.vector(), b)

# save solution
File(WD_PATH + 'plots/pv1_1dlim/pressure1d.pvd') << uh1d

visualize(mesh1d, uh1d)

For $N$ branches over $M$ endpoints $E_1, E_2, \cdots, E_M$, we have
$$
\int_{\Omega_{1D}} \frac{\partial p_{1D}}{\partial s} \frac{\partial v_{1D}}{\partial s} \, ds + \sum_{j=1}^{M} \gamma v_{1D}(E_j) (p_{\infty} - p_{1D}(E_j)) = \beta \int_{\Omega_{1D}} (\overline{p_{3D}} - p_{1D}) v_{1D} \, ds
$$

So the new formulation becomes
$$
a_{11} = \alpha_{1} \int_{\Omega_{1D}} \frac{\partial u_{1D}}{\partial s} \frac{\partial v_{1D}}{\partial s} \, ds + \beta \int_{\Omega_{1D}} u_{1D} v_{1D} \, ds + \sum_{j=1}^{M} \gamma v_{1D}(E_j) u_{1D}(E_j)
$$

The right-hand side is
$$
L_{1} = \beta \int_{\Omega_{1D}} \overline{p_{3D}} v_{1D} \, ds + \sum_{j=1}^{M} \gamma v_{1D}(E_j) p_{\infty}
$$

In [ ]:
# @title 1D domain analysis only with Robin boundaries
import copy

# Function space
V1 = FunctionSpace(mesh1d, "CG", 1)
u1 = TrialFunction(V1)
v1 = TestFunction(V1)

# Constants for simulation
alpha1 = Constant(1.45e4)
beta = Constant(3.09e-5)
gamma = Constant(0.2)  # Adjust gamma
p_infty = Constant(9.0)  # Far-field pressure

debug_x_list = []

# Custom UserExpression for Robin boundary condition
class RobinBC(UserExpression):
    def __init__(self, gamma, p_infty, **kwargs):
        super().__init__(**kwargs)
        self.gamma = gamma
        self.p_infty = p_infty

    def eval(self, values, x):
        # print(f"eval called! self: {self} values: {values} x: {x}")
        debug_x_list.append(copy.deepcopy(x))
        values[0] = self.gamma * self.p_infty

    def value_shape(self):
        return ()

# Create instance of RobinBC
robin_bc = RobinBC(gamma=gamma, p_infty=p_infty, degree=0)

# Weak form
a11 = alpha1 * inner(grad(u1), grad(v1)) * dx + beta * inner(u1, v1) * dx + robin_bc * inner(u1, v1) * ds
L1 = beta * inner(p_infty, v1) * dx + robin_bc * inner(p_infty, v1) * ds

# Assemble system
A, b = assemble_system(a11, L1)

# Solve system
uh1d = Function(V1)
solve(A, uh1d.vector(), b)

# Save solution
File(WD_PATH + 'plots/pv1_mod/pressure1d.pvd') << uh1d

visualize(mesh1d, uh1d)

In [ ]:
# @title pv_1 robin coupling with 3D
import json
import pandas as pd
from dolfin import *
from xii import *
import numpy as np
from graphnics import *
import matplotlib.pyplot as plt

mesh3d = UnitCubeMesh(16, 16, 32)

# fit 3D mesh around 1D mesh
c = mesh3d.coordinates()
xl, yl, zl = (np.max(node_coords, axis=0) - np.min(node_coords, axis=0))  # graph length scales
c[:, :] *= [xl + 3, yl + 3, zl]  # Rescale lengths

# set constants for simulation
Alpha1 = Constant(9.6e-2)
alpha1 = Constant(1.45e4)
beta = Constant(3.09e-5)
gamma = Constant(0.2)  # Adjust gamma
p_infty = Constant(9.0)  # Far-field pressure

# set boundary conditions for simulation
bc_3d = Constant(3)

# define function that returns lateral faces of 3D boundary
def boundary_3d(x, on_boundary):
    return on_boundary and not near(x[2], 0) and not near(x[2], zl)

# pressure space on global mesh
V3 = FunctionSpace(mesh3d, "CG", 1)
V1 = FunctionSpace(mesh1d, "CG", 1)
W = [V3, V1]

u3, u1 = list(map(TrialFunction, W))
v3, v1 = list(map(TestFunction, W))

# Custom UserExpression for Robin boundary condition
class RobinBC(UserExpression):
    def __init__(self, gamma, p_infty, **kwargs):
        super().__init__(**kwargs)
        self.gamma = gamma
        self.p_infty = p_infty

    def eval(self, values, x):
        # print(f"eval called! self: {self} values: {values} x: {x}")
        # debug_x_list.append(copy.deepcopy(x))
        values[0] = self.gamma * self.p_infty

    def value_shape(self):
        return ()

# create a radius function for the averaging surface
class RadiusFunction(UserExpression):
    def __init__(self, radii_map, pos_map, **kwargs):
        self.radii_map = radii_map
        self.pos_map = pos_map
        super().__init__(**kwargs)

    def eval(self, value, x):
        min_dist = float('inf')
        closest_radius = 0
        for node, position in self.pos_map.items():
            pos = np.array(position)
            dist = np.linalg.norm(x - pos)
            if dist < min_dist:
                min_dist = dist
                closest_radius = self.radii_map[node]
        value[0] = closest_radius

    def value_shape(self):
        return ()

robin_bc = RobinBC(gamma=gamma, p_infty=p_infty, degree=0)

radius_function = RadiusFunction(radii, pos, degree=2)
cylinder = Circle(radius=radius_function, degree=10)

Pi_u = Average(u3, mesh1d, cylinder)
Pi_v = Average(v3, mesh1d, cylinder)

# Dirac measure
dxGamma = Measure("dx", domain=mesh1d)

# blocks
a00 = Alpha1 * inner(grad(u3), grad(v3)) * dx + beta * inner(Pi_u, Pi_v) * dxGamma
a01 = -beta * inner(u1, Pi_v) * dxGamma
a10 = -beta * inner(Pi_u, v1) * dxGamma
a11 = alpha1 * inner(grad(u1), grad(v1)) * dx + beta * inner(u1, v1) * dxGamma

# right-hand side
L0 = inner(Constant(0), Pi_v) * dxGamma
L1 = inner(Constant(0), v1) * dxGamma

a11 += robin_bc * inner(u1, v1) * ds
L1 += beta * inner(p_infty, v1) * dx + robin_bc * inner(p_infty, v1) * ds

a = [[a00, a01], [a10, a11]]
L = [L0, L1]

W_bcs = [[DirichletBC(V3, bc_3d, boundary_3d)], []]

A, b = map(ii_assemble, (a, L))
A, b = apply_bc(A, b, W_bcs)
A, b = map(ii_convert, (A, b))

wh = ii_Function(W)
solver = LUSolver(A, "mumps")
solver.solve(wh.vector(), b)

uh3d, uh1d = wh
File(WD_PATH + 'plots/pv1_mod2/pressure1d.pvd') << uh1d
File(WD_PATH + 'plots/pv1_mod2/pressure3d.pvd') << uh3d

visualize(mesh1d, uh1d, mesh3d=mesh3d, uh3d=uh3d)

In [ ]:
# @title .vtk reader
import vtk
import pandas as pd
from vtk.util.numpy_support import vtk_to_numpy

# Function to read .vtk file and extract required information
def read_vtk_file(file_path):
    # Read the .vtk file
    reader = vtk.vtkPolyDataReader()
    reader.SetFileName(file_path)
    reader.Update()

    # Extract the unstructured grid
    output = reader.GetOutput()

    # Extract point coordinates
    points = vtk_to_numpy(output.GetPoints().GetData())

    # Extract connectivity information
    connectivity = []
    for i in range(output.GetNumberOfCells()):
        cell = output.GetCell(i)
        connectivity.append([cell.GetPointId(j) for j in range(cell.GetNumberOfPoints())])

    # Extract data arrays
    damage = vtk_to_numpy(output.GetPointData().GetArray("Damage"))
    radius = vtk_to_numpy(output.GetPointData().GetArray("Radius"))

    # Create DataFrame
    df_points = pd.DataFrame(points, columns=['x', 'y', 'z'])
    df_points['Damage'] = damage
    df_points['Radius'] = radius

    # Create DataFrame for connectivity
    df_connectivity = pd.DataFrame(connectivity, columns=['point1', 'point2'])

    return df_points, df_connectivity

# Example usage
file_path = WD_PATH + 'data/vtk/sortedDomain.vtk'
df_points, df_connectivity = read_vtk_file(file_path)

In [ ]:
# @title pv_1 modified using .vtk conversion
import pandas as pd
from dolfin import *
from xii import *
import numpy as np
from graphnics import *
import matplotlib.pyplot as plt

# Initialize the FenicsGraph
G = FenicsGraph()
ind = 0
branch_points = {}

# Read DataFrames instead of JSON files
for n, df in enumerate(centerline_dfs):
    # Get coords + radius at each point
    points = df[['x', 'y', 'z']].values.tolist()
    radius = df['radius'].tolist()
    G.add_nodes_from(range(ind - n, ind + len(points) - n))

    # Check if first coord is branch point from previous centerlines
    v1 = 0
    for key, val in branch_points.items():
        if points[0] == val:
            v1 = key
            break

    # Add coords and radius to nodes Fenics graph
    v2 = ind - n + 1
    pos_v1 = points[0]
    pos_v2 = points[1]
    G.nodes[v1]["pos"] = pos_v1
    G.nodes[v2]["pos"] = pos_v2
    G.nodes[v1]["radius"] = radius[0]
    G.nodes[v2]["radius"] = radius[1]
    G.add_edge(v1, v2)

    for i in range(len(points) - 2):
        v1 = ind - n + 1 + i
        v2 = v1 + 1
        pos_v1 = points[i + 1]
        pos_v2 = points[i + 2]
        G.nodes[v1]["pos"] = pos_v1
        G.nodes[v2]["pos"] = pos_v2
        G.nodes[v1]["radius"] = radius[i + 1]
        G.nodes[v2]["radius"] = radius[i + 2]
        G.add_edge(v1, v2)

    # Store last point as a branch point
    ind += len(points)
    branch_points.update({ind - n - 1: pos_v2})

# Create 1D mesh
G.make_mesh()
mesh1d = G.mesh

# Get positions of 1D mesh
pos = nx.get_node_attributes(G, "pos")
radii = nx.get_node_attributes(G, "radius")
node_coords = np.asarray(list(pos.values()))
xmin, ymin, zmin = np.min(node_coords, axis=0)

# Recenter 1D coordinates to all be >= 0
d = mesh1d.coordinates()
d[:, :] += [-xmin, -ymin, -zmin]

# Create 3D mesh
mesh3d = UnitCubeMesh(16, 16, 32)

# Fit 3D mesh around 1D mesh
c = mesh3d.coordinates()
xl, yl, zl = (np.max(node_coords, axis=0) - np.min(node_coords, axis=0))  # graph length scales
c[:, :] *= [xl + 3, yl + 3, zl]  # Rescale lengths

# Set constants for simulation
Alpha1 = Constant(9.6e-2)
alpha1 = Constant(1.45e4)
beta = Constant(3.09e-5)

# Set boundary conditions for simulation
bc_3d = Constant(3)
bc_1d = Expression("0.02*x[2]+6", degree=0)

# Define function that returns lateral faces of 3D boundary
def boundary_3d(x, on_boundary):
    return on_boundary and not near(x[2], 0) and not near(x[2], zl)

# Pressure space on global mesh
V3 = FunctionSpace(mesh3d, "CG", 1)
V1 = FunctionSpace(mesh1d, "CG", 1)
W = [V3, V1]

u3, u1 = list(map(TrialFunction, W))
v3, v1 = list(map(TestFunction, W))

# Create a radius function for the averaging surface
class RadiusFunction(UserExpression):
    def __init__(self, radii_map, pos_map, **kwargs):
        self.radii_map = radii_map
        self.pos_map = pos_map
        super().__init__(**kwargs)

    def eval(self, value, x):
        min_dist = float('inf')
        closest_radius = 0
        for node, position in self.pos_map.items():
            pos = np.array(position)
            dist = np.linalg.norm(x - pos)
            if dist < min_dist:
                min_dist = dist
                closest_radius = self.radii_map[node]
        value[0] = closest_radius

    def value_shape(self):
        return ()

radius_function = RadiusFunction(radii, pos, degree=2)
cylinder = Circle(radius=radius_function, degree=10)

Pi_u = Average(u3, mesh1d, cylinder)
Pi_v = Average(v3, mesh1d, cylinder)

# Dirac measure
dxGamma = Measure("dx", domain=mesh1d)

# Blocks
a00 = Alpha1 * inner(grad(u3), grad(v3)) * dx + beta * inner(Pi_u, Pi_v) * dxGamma
a01 = -beta * inner(u1, Pi_v) * dxGamma
a10 = -beta * inner(Pi_u, v1) * dxGamma
a11 = alpha1 * inner(grad(u1), grad(v1)) * dx + beta * inner(u1, v1) * dxGamma

# Right-hand side
L0 = inner(Constant(0), Pi_v) * dxGamma
L1 = inner(Constant(0), v1) * dxGamma

a = [[a00, a01], [a10, a11]]
L = [L0, L1]

W_bcs = [[DirichletBC(V3, bc_3d, boundary_3d)], []] #DirichletBC(V1, bc_1d, "on_boundary")

A, b = map(ii_assemble, (a, L))
A, b = apply_bc(A, b, W_bcs)
A, b = map(ii_convert, (A, b))

wh = ii_Function(W)
solver = LUSolver(A, "mumps")
solver.solve(wh.vector(), b)

uh3d, uh1d = wh
uh3d.rename("Pressure_3D", "label_3D")
uh1d.rename("Pressure_1D", "label_1D")
File(WD_PATH + 'plots/pv1_mod_2/pressure1d.pvd') << uh1d
File(WD_PATH + 'plots/pv1_mod_2/pressure3d.pvd') << uh3d

In [ ]:
# @title pv_1 modified over serena data
import json
import pandas as pd
from dolfin import *
from xii import *
import numpy as np
from graphnics import *
import matplotlib.pyplot as plt

# function to load JSON data into a DataFrame
def json_to_dataframe(file_path):
    with open(file_path) as f:
        data = json.load(f)
    points = data['markups'][0]['controlPoints']
    radius = data['markups'][0]['measurements'][3]['controlPointValues']
    coords = [point['position'] for point in points]
    df = pd.DataFrame(coords, columns=['x', 'y', 'z'])
    df['radius'] = radius
    return df

# convert all JSON files into df and store them in an array
dataframes = []
for n in range(29):
    file_name = WD_PATH + '/data/pv_json1_mod/Centerline_' + str(n) + '.mrk.json'
    df = json_to_dataframe(file_name)
    dataframes.append(df)

G = FenicsGraph()
ind = 0
branch_points = {}

for n, df in enumerate(dataframes):
    # Get coords + radius at each point
    points = df[['x', 'y', 'z']].values.tolist()
    radius = df['radius'].tolist()
    G.add_nodes_from(range(ind - n, ind + len(points) - n))

    # Check if first coord is branch point from previous centerlines
    v1 = 0
    for key, val in branch_points.items():
        if points[0] == val:
            v1 = key
            break

    # Add coords and radius to nodes Fenics graph
    v2 = ind - n + 1
    pos_v1 = points[0]
    pos_v2 = points[1]
    G.nodes[v1]["pos"] = pos_v1
    G.nodes[v2]["pos"] = pos_v2
    G.nodes[v1]["radius"] = radius[0]
    G.nodes[v2]["radius"] = radius[1]
    G.add_edge(v1, v2)

    for i in range(len(points) - 2):
        v1 = ind - n + 1 + i
        v2 = v1 + 1
        pos_v1 = points[i + 1]
        pos_v2 = points[i + 2]
        G.nodes[v1]["pos"] = pos_v1
        G.nodes[v2]["pos"] = pos_v2
        G.nodes[v1]["radius"] = radius[i + 1]
        G.nodes[v2]["radius"] = radius[i + 2]
        G.add_edge(v1, v2)

    # Store last point as a branch point
    ind += len(points)
    branch_points.update({ind - n - 1: pos_v2})

# create 1D mesh
G.make_mesh()
mesh1d = G.mesh

# get positions of 1D mesh
pos = nx.get_node_attributes(G, "pos")
radii = nx.get_node_attributes(G, "radius")
node_coords = np.asarray(list(pos.values()))
xmin, ymin, zmin = np.min(node_coords, axis=0)

# recenter 1D coordinates to all be >= 0
d = mesh1d.coordinates()
d[:, :] += [-xmin, -ymin, -zmin]

mesh3d = UnitCubeMesh(16, 16, 32)

# fit 3D mesh around 1D mesh
c = mesh3d.coordinates()
xl, yl, zl = (np.max(node_coords, axis=0) - np.min(node_coords, axis=0))  # graph length scales
c[:, :] *= [xl + 3, yl + 3, zl]  # Rescale lengths

# set constants for simulation
Alpha1 = Constant(9.6e-2)
alpha1 = Constant(1.45e4)
beta = Constant(3.09e-5)

# set boundary conditions for simulation
bc_3d = Constant(3)
bc_1d = Expression("0.02*x[2]+6", degree=0)

# define function that returns lateral faces of 3D boundary
def boundary_3d(x, on_boundary):
    return on_boundary and not near(x[2], 0) and not near(x[2], zl)

# pressure space on global mesh
V3 = FunctionSpace(mesh3d, "CG", 1)
V1 = FunctionSpace(mesh1d, "CG", 1)
W = [V3, V1]

u3, u1 = list(map(TrialFunction, W))
v3, v1 = list(map(TestFunction, W))

# create a radius function for the averaging surface
class RadiusFunction(UserExpression):
    def __init__(self, radii_map, pos_map, **kwargs):
        self.radii_map = radii_map
        self.pos_map = pos_map
        super().__init__(**kwargs)

    def eval(self, value, x):
        min_dist = float('inf')
        closest_radius = 0
        for node, position in self.pos_map.items():
            pos = np.array(position)
            dist = np.linalg.norm(x - pos)
            if dist < min_dist:
                min_dist = dist
                closest_radius = self.radii_map[node]
        value[0] = closest_radius

    def value_shape(self):
        return ()

radius_function = RadiusFunction(radii, pos, degree=2)
cylinder = Circle(radius=radius_function, degree=10)

Pi_u = Average(u3, mesh1d, cylinder)
Pi_v = Average(v3, mesh1d, cylinder)

# Dirac measure
dxGamma = Measure("dx", domain=mesh1d)

# blocks
a00 = Alpha1 * inner(grad(u3), grad(v3)) * dx + beta * inner(Pi_u, Pi_v) * dxGamma
a01 = -beta * inner(u1, Pi_v) * dxGamma
a10 = -beta * inner(Pi_u, v1) * dxGamma
a11 = alpha1 * inner(grad(u1), grad(v1)) * dx + beta * inner(u1, v1) * dxGamma

# right-hand side
L0 = inner(Constant(0), Pi_v) * dxGamma
L1 = inner(Constant(0), v1) * dxGamma

a = [[a00, a01], [a10, a11]]
L = [L0, L1]

W_bcs = [[DirichletBC(V3, bc_3d, boundary_3d)], [DirichletBC(V1, bc_1d, "on_boundary")]]

A, b = map(ii_assemble, (a, L))
A, b = apply_bc(A, b, W_bcs)
A, b = map(ii_convert, (A, b))

wh = ii_Function(W)
solver = LUSolver(A, "mumps")
solver.solve(wh.vector(), b)

uh3d, uh1d = wh
File(WD_PATH + 'plots/pv1_mod/pressure1d.pvd') << uh1d
File(WD_PATH + 'plots/pv1_mod/pressure3d.pvd') << uh3d